In [1]:
# train with fewer data. ideal data
import os
import keras
import pandas as pd
import numpy as np
from keras import layers
from keras import regularizers
from sklearn import preprocessing
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import RMSprop, SGD, Adam
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint, Callback
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp

2023-03-03 10:33:17.452478: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 10:33:18.157088: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda-10.0/lib64:/home/rilab/DTran/.other/Cpp_libraries/librealsense-2.47.0/install/lib:/home/rilab/anaconda3/envs/huiEnv/lib/
2023-03-03 10:33:18.157159: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No

In [2]:
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
e_data = pd.read_csv("encoded_data_64.csv")
label = pd.read_csv("properity.csv")

conditions = ["materials", "color", "distance", "mode"]

In [4]:
#r_data = r_data.drop(columns = conditions)
#r_data = pd.concat([label, r_data], axis = 1)

#e_data = e_data.drop(columns = conditions)
e_data = e_data.drop(columns = "materials")

e_data = pd.concat([label, e_data], axis = 1)

e_data = e_data.drop(columns = ["Hardness", "Rigidity", "Strength"])

In [5]:
e_data

,Density,names,spectrum_0,spectrum_1,spectrum_2,spectrum_3,spectrum_4,spectrum_5,spectrum_6,spectrum_7,...,spectrum_54,spectrum_55,spectrum_56,spectrum_57,spectrum_58,spectrum_59,spectrum_60,spectrum_61,spectrum_62,spectrum_63
0,0.7,Cardboard,0.068472,0.000798,0.001004,0.001597,0.001190,0.001387,0.001140,0.001251,...,0.001410,0.001230,0.001109,0.001239,0.002353,0.013542,0.000959,0.001235,0.001654,0.001062
1,0.7,Cardboard,0.069635,0.000800,0.001007,0.001591,0.001185,0.001386,0.001142,0.001244,...,0.001401,0.001227,0.001113,0.001243,0.002366,0.013688,0.000963,0.001240,0.001661,0.001065
2,0.7,Cardboard,0.069555,0.000802,0.001009,0.001596,0.001189,0.001389,0.001144,0.001248,...,0.001405,0.001230,0.001115,0.001245,0.002369,0.013689,0.000965,0.001242,0.001664,0.001067
3,0.7,Cardboard,0.068797,0.000795,0.001000,0.001586,0.001182,0.001379,0.001135,0.001241,...,0.001398,0.001222,0.001105,0.001234,0.002347,0.013557,0.000956,0.001231,0.001649,0.001058
4,0.7,Cardboard,0.069200,0.000808,0.001017,0.001614,0.001203,0.001403,0.001154,0.001265,...,0.001424,0.001244,0.001123,0.001254,0.002381,0.013690,0.000971,0.001250,0.001674,0.001075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,0.8,Wood,0.177916,0.001669,0.002065,0.002501,0.001866,0.002422,0.002268,0.001885,...,0.001998,0.002049,0.002333,0.002529,0.005113,0.031733,0.002108,0.002574,0.003430,0.002144
1256,0.8,Wood,0.171001,0.001768,0.002182,0.002713,0.002029,0.002593,0.002403,0.002062,...,0.002193,0.002215,0.002454,0.002665,0.005292,0.031593,0.002209,0.002700,0.003582,0.002268
1257,0.8,Wood,0.068658,0.018290,0.019366,0.061013,0.049947,0.036245,0.025003,0.067426,...,0.078777,0.044816,0.018668,0.021073,0.024119,0.041496,0.015430,0.019119,0.020862,0.021492
1258,0.8,Wood,0.042997,0.013600,0.014174,0.054635,0.044808,0.029736,0.019006,0.063586,...,0.077216,0.038426,0.013364,0.015166,0.016858,0.027898,0.010784,0.013917,0.014934,0.015921


In [6]:
# normalize data 
tep = e_data.loc[:, [ "names", "Density"]]
e_data_sp = e_data.copy().drop(columns = ['names', "Density"])
e_data_sp

,spectrum_0,spectrum_1,spectrum_2,spectrum_3,spectrum_4,spectrum_5,spectrum_6,spectrum_7,spectrum_8,spectrum_9,...,spectrum_54,spectrum_55,spectrum_56,spectrum_57,spectrum_58,spectrum_59,spectrum_60,spectrum_61,spectrum_62,spectrum_63
0,0.068472,0.000798,0.001004,0.001597,0.001190,0.001387,0.001140,0.001251,0.004277,0.001168,...,0.001410,0.001230,0.001109,0.001239,0.002353,0.013542,0.000959,0.001235,0.001654,0.001062
1,0.069635,0.000800,0.001007,0.001591,0.001185,0.001386,0.001142,0.001244,0.004314,0.001159,...,0.001401,0.001227,0.001113,0.001243,0.002366,0.013688,0.000963,0.001240,0.001661,0.001065
2,0.069555,0.000802,0.001009,0.001596,0.001189,0.001389,0.001144,0.001248,0.004316,0.001163,...,0.001405,0.001230,0.001115,0.001245,0.002369,0.013689,0.000965,0.001242,0.001664,0.001067
3,0.068797,0.000795,0.001000,0.001586,0.001182,0.001379,0.001135,0.001241,0.004274,0.001158,...,0.001398,0.001222,0.001105,0.001234,0.002347,0.013557,0.000956,0.001231,0.001649,0.001058
4,0.069200,0.000808,0.001017,0.001614,0.001203,0.001403,0.001154,0.001265,0.004328,0.001180,...,0.001424,0.001244,0.001123,0.001254,0.002381,0.013690,0.000971,0.001250,0.001674,0.001075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,0.177916,0.001669,0.002065,0.002501,0.001866,0.002422,0.002268,0.001885,0.010040,0.001635,...,0.001998,0.002049,0.002333,0.002529,0.005113,0.031733,0.002108,0.002574,0.003430,0.002144
1256,0.171001,0.001768,0.002182,0.002713,0.002029,0.002593,0.002403,0.002062,0.010187,0.001808,...,0.002193,0.002215,0.002454,0.002665,0.005292,0.031593,0.002209,0.002700,0.003582,0.002268
1257,0.068658,0.018290,0.019366,0.061013,0.049947,0.036245,0.025003,0.067426,0.026343,0.087812,...,0.078777,0.044816,0.018668,0.021073,0.024119,0.041496,0.015430,0.019119,0.020862,0.021492
1258,0.042997,0.013600,0.014174,0.054635,0.044808,0.029736,0.019006,0.063586,0.018080,0.088403,...,0.077216,0.038426,0.013364,0.015166,0.016858,0.027898,0.010784,0.013917,0.014934,0.015921


In [7]:
e_data_sp = (e_data_sp - e_data_sp.mean())/e_data_sp.std()
e_data_sp

,spectrum_0,spectrum_1,spectrum_2,spectrum_3,spectrum_4,spectrum_5,spectrum_6,spectrum_7,spectrum_8,spectrum_9,...,spectrum_54,spectrum_55,spectrum_56,spectrum_57,spectrum_58,spectrum_59,spectrum_60,spectrum_61,spectrum_62,spectrum_63
0,-1.115912,-0.728643,-0.749686,-0.736475,-0.715072,-0.743912,-0.749965,-0.716269,-0.981183,-0.710028,...,-0.717374,-0.724037,-0.766685,-0.776774,-0.872160,-1.148111,-0.777188,-0.768012,-0.804966,-0.751950
1,-1.107021,-0.728564,-0.749586,-0.736574,-0.715155,-0.743936,-0.749900,-0.716370,-0.980187,-0.710138,...,-0.717494,-0.724098,-0.766545,-0.776651,-0.871762,-1.145388,-0.777012,-0.767850,-0.804733,-0.751868
2,-1.107631,-0.728518,-0.749531,-0.736507,-0.715099,-0.743868,-0.749841,-0.716316,-0.980121,-0.710092,...,-0.717439,-0.724039,-0.766487,-0.776585,-0.871679,-1.145386,-0.776958,-0.767793,-0.804666,-0.751810
3,-1.113432,-0.728751,-0.749813,-0.736657,-0.715224,-0.744087,-0.750108,-0.716417,-0.981245,-0.710159,...,-0.717527,-0.724194,-0.766816,-0.776929,-0.872326,-1.147828,-0.777308,-0.768141,-0.805111,-0.752090
4,-1.110347,-0.728303,-0.749277,-0.736200,-0.714843,-0.743558,-0.749570,-0.716072,-0.979809,-0.709883,...,-0.717189,-0.723770,-0.766220,-0.776280,-0.871300,-1.145354,-0.776703,-0.767523,-0.804350,-0.751541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,-0.279136,-0.699879,-0.715272,-0.721783,-0.702883,-0.720305,-0.718238,-0.706856,-0.827207,-0.704335,...,-0.709624,-0.708081,-0.725917,-0.734978,-0.789022,-0.810365,-0.732375,-0.724863,-0.749657,-0.718353
1256,-0.332007,-0.696639,-0.711489,-0.718338,-0.699946,-0.716421,-0.714437,-0.704214,-0.823262,-0.702229,...,-0.707049,-0.704845,-0.721901,-0.730574,-0.783617,-0.812966,-0.728413,-0.720826,-0.744929,-0.714494
1257,-1.114494,-0.151241,-0.154344,0.229621,0.164377,0.050968,-0.078866,0.267358,-0.391611,0.345947,...,0.302501,0.125445,-0.181908,-0.134064,-0.216604,-0.629094,-0.212757,-0.191827,-0.206766,-0.117266
1258,-1.310687,-0.306069,-0.322686,0.125911,0.071682,-0.097453,-0.247510,0.210284,-0.612372,0.353154,...,0.281922,0.000922,-0.358556,-0.325485,-0.435267,-0.881566,-0.393971,-0.359431,-0.391404,-0.290328


In [8]:
e_data = pd.concat([tep, e_data_sp], axis =1)
e_data

,names,Density,spectrum_0,spectrum_1,spectrum_2,spectrum_3,spectrum_4,spectrum_5,spectrum_6,spectrum_7,...,spectrum_54,spectrum_55,spectrum_56,spectrum_57,spectrum_58,spectrum_59,spectrum_60,spectrum_61,spectrum_62,spectrum_63
0,Cardboard,0.7,-1.115912,-0.728643,-0.749686,-0.736475,-0.715072,-0.743912,-0.749965,-0.716269,...,-0.717374,-0.724037,-0.766685,-0.776774,-0.872160,-1.148111,-0.777188,-0.768012,-0.804966,-0.751950
1,Cardboard,0.7,-1.107021,-0.728564,-0.749586,-0.736574,-0.715155,-0.743936,-0.749900,-0.716370,...,-0.717494,-0.724098,-0.766545,-0.776651,-0.871762,-1.145388,-0.777012,-0.767850,-0.804733,-0.751868
2,Cardboard,0.7,-1.107631,-0.728518,-0.749531,-0.736507,-0.715099,-0.743868,-0.749841,-0.716316,...,-0.717439,-0.724039,-0.766487,-0.776585,-0.871679,-1.145386,-0.776958,-0.767793,-0.804666,-0.751810
3,Cardboard,0.7,-1.113432,-0.728751,-0.749813,-0.736657,-0.715224,-0.744087,-0.750108,-0.716417,...,-0.717527,-0.724194,-0.766816,-0.776929,-0.872326,-1.147828,-0.777308,-0.768141,-0.805111,-0.752090
4,Cardboard,0.7,-1.110347,-0.728303,-0.749277,-0.736200,-0.714843,-0.743558,-0.749570,-0.716072,...,-0.717189,-0.723770,-0.766220,-0.776280,-0.871300,-1.145354,-0.776703,-0.767523,-0.804350,-0.751541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,Wood,0.8,-0.279136,-0.699879,-0.715272,-0.721783,-0.702883,-0.720305,-0.718238,-0.706856,...,-0.709624,-0.708081,-0.725917,-0.734978,-0.789022,-0.810365,-0.732375,-0.724863,-0.749657,-0.718353
1256,Wood,0.8,-0.332007,-0.696639,-0.711489,-0.718338,-0.699946,-0.716421,-0.714437,-0.704214,...,-0.707049,-0.704845,-0.721901,-0.730574,-0.783617,-0.812966,-0.728413,-0.720826,-0.744929,-0.714494
1257,Wood,0.8,-1.114494,-0.151241,-0.154344,0.229621,0.164377,0.050968,-0.078866,0.267358,...,0.302501,0.125445,-0.181908,-0.134064,-0.216604,-0.629094,-0.212757,-0.191827,-0.206766,-0.117266
1258,Wood,0.8,-1.310687,-0.306069,-0.322686,0.125911,0.071682,-0.097453,-0.247510,0.210284,...,0.281922,0.000922,-0.358556,-0.325485,-0.435267,-0.881566,-0.393971,-0.359431,-0.391404,-0.290328


In [9]:
test_list = ["Cardboard1", "Ceramic1", "Glass1", "Plastic1", "Rock1"]
e_test = e_data[(e_data["names"]=="Cardboard1" )|(e_data["names"]=="Ceramic1" )|(e_data["names"]=="Glass1") |(e_data["names"]=="Plastic1") |(e_data["names"]=="Rock1")].reset_index(drop=True)
e_test

,names,Density,spectrum_0,spectrum_1,spectrum_2,spectrum_3,spectrum_4,spectrum_5,spectrum_6,spectrum_7,...,spectrum_54,spectrum_55,spectrum_56,spectrum_57,spectrum_58,spectrum_59,spectrum_60,spectrum_61,spectrum_62,spectrum_63
0,Cardboard1,0.7,-1.222089,-0.731775,-0.753505,-0.737678,-0.716104,-0.746348,-0.753370,-0.716866,...,-0.717742,-0.725527,-0.771206,-0.781378,-0.881510,-1.188793,-0.782123,-0.772902,-0.811379,-0.755607
1,Cardboard1,0.7,-1.222889,-0.732071,-0.753856,-0.738047,-0.716408,-0.746739,-0.753741,-0.717155,...,-0.718031,-0.725854,-0.771593,-0.781812,-0.882156,-1.190055,-0.782506,-0.773298,-0.811857,-0.755978
2,Cardboard1,0.7,-1.223594,-0.732304,-0.754133,-0.738336,-0.716645,-0.747047,-0.754034,-0.717382,...,-0.718257,-0.726112,-0.771899,-0.782154,-0.882668,-1.191074,-0.782807,-0.773610,-0.812235,-0.756272
3,Cardboard1,0.7,-1.253176,-0.715537,-0.734651,-0.712910,-0.695747,-0.722589,-0.732372,-0.696245,...,-0.696845,-0.704232,-0.750996,-0.758113,-0.850779,-1.152454,-0.762656,-0.752706,-0.787926,-0.735098
4,Cardboard1,0.7,-1.286577,-0.697994,-0.714912,-0.681122,-0.669507,-0.694998,-0.709313,-0.667894,...,-0.667802,-0.677601,-0.730416,-0.733973,-0.822202,-1.129839,-0.743140,-0.732515,-0.765458,-0.713138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,Rock1,3.2,-0.890006,-0.713101,-0.731134,-0.724364,-0.704868,-0.728169,-0.732157,-0.707552,...,-0.709225,-0.712104,-0.745627,-0.754517,-0.833966,-1.030883,-0.755103,-0.745954,-0.777340,-0.733455
446,Rock1,3.2,-0.885503,-0.712588,-0.730527,-0.723925,-0.704497,-0.727629,-0.731567,-0.707226,...,-0.708916,-0.711679,-0.744948,-0.753794,-0.832799,-1.027887,-0.754397,-0.745252,-0.776474,-0.732846
447,Rock1,3.2,-1.431018,-0.394084,-0.422334,0.090835,0.049914,-0.179356,-0.347860,0.228330,...,0.348304,-0.056555,-0.464889,-0.444711,-0.573235,-1.041221,-0.502578,-0.458578,-0.503277,-0.395475
448,Rock1,3.2,-1.481978,-0.463948,-0.494377,0.003633,-0.032057,-0.266088,-0.426501,0.152261,...,0.285322,-0.143506,-0.535887,-0.521557,-0.652785,-1.120802,-0.571369,-0.528830,-0.576704,-0.471381


In [10]:
e_train = e_data[(e_data["names"]!="Cardboard1" )&(e_data["names"]!="Ceramic1" )&(e_data["names"]!="Glass1")&(e_data["names"]!="Plastic1") &(e_data["names"]!="Rock1")].reset_index(drop=True)
e_train

,names,Density,spectrum_0,spectrum_1,spectrum_2,spectrum_3,spectrum_4,spectrum_5,spectrum_6,spectrum_7,...,spectrum_54,spectrum_55,spectrum_56,spectrum_57,spectrum_58,spectrum_59,spectrum_60,spectrum_61,spectrum_62,spectrum_63
0,Cardboard,0.7,-1.115912,-0.728643,-0.749686,-0.736475,-0.715072,-0.743912,-0.749965,-0.716269,...,-0.717374,-0.724037,-0.766685,-0.776774,-0.872160,-1.148111,-0.777188,-0.768012,-0.804966,-0.751950
1,Cardboard,0.7,-1.107021,-0.728564,-0.749586,-0.736574,-0.715155,-0.743936,-0.749900,-0.716370,...,-0.717494,-0.724098,-0.766545,-0.776651,-0.871762,-1.145388,-0.777012,-0.767850,-0.804733,-0.751868
2,Cardboard,0.7,-1.107631,-0.728518,-0.749531,-0.736507,-0.715099,-0.743868,-0.749841,-0.716316,...,-0.717439,-0.724039,-0.766487,-0.776585,-0.871679,-1.145386,-0.776958,-0.767793,-0.804666,-0.751810
3,Cardboard,0.7,-1.113432,-0.728751,-0.749813,-0.736657,-0.715224,-0.744087,-0.750108,-0.716417,...,-0.717527,-0.724194,-0.766816,-0.776929,-0.872326,-1.147828,-0.777308,-0.768141,-0.805111,-0.752090
4,Cardboard,0.7,-1.110347,-0.728303,-0.749277,-0.736200,-0.714843,-0.743558,-0.749570,-0.716072,...,-0.717189,-0.723770,-0.766220,-0.776280,-0.871300,-1.145354,-0.776703,-0.767523,-0.804350,-0.751541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,Wood,0.8,-0.279136,-0.699879,-0.715272,-0.721783,-0.702883,-0.720305,-0.718238,-0.706856,...,-0.709624,-0.708081,-0.725917,-0.734978,-0.789022,-0.810365,-0.732375,-0.724863,-0.749657,-0.718353
806,Wood,0.8,-0.332007,-0.696639,-0.711489,-0.718338,-0.699946,-0.716421,-0.714437,-0.704214,...,-0.707049,-0.704845,-0.721901,-0.730574,-0.783617,-0.812966,-0.728413,-0.720826,-0.744929,-0.714494
807,Wood,0.8,-1.114494,-0.151241,-0.154344,0.229621,0.164377,0.050968,-0.078866,0.267358,...,0.302501,0.125445,-0.181908,-0.134064,-0.216604,-0.629094,-0.212757,-0.191827,-0.206766,-0.117266
808,Wood,0.8,-1.310687,-0.306069,-0.322686,0.125911,0.071682,-0.097453,-0.247510,0.210284,...,0.281922,0.000922,-0.358556,-0.325485,-0.435267,-0.881566,-0.393971,-0.359431,-0.391404,-0.290328


In [11]:
e_test_name = e_test["names"].to_numpy()
e_test = e_test.drop(columns = ["names"])
e_train = e_train.drop(columns = ["names"])

In [12]:
e_test.shape[0]

450

In [13]:
e_test_y = e_test["Density"]
e_test_x = e_test.drop(columns = ["Density"])

In [14]:
e_train_y = e_train["Density"]
e_train_x = e_train.drop(columns = ["Density"])

In [15]:
x_tra, x_val, y_tra, y_val = train_test_split(e_train_x, e_train_y, test_size=0.15, random_state=50)

In [16]:
train_size = x_tra.shape[0]

In [17]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [18]:
BATCH_SIZE = 2
EPOCHS = 10000
learning_rate = 0.0001
encoding_dim = 64

def bnn():
    model = Sequential()
    model.add(tfp.layers.DenseVariational(
            units=encoding_dim,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        ))

    model.add(tfp.layers.DenseVariational(
            units=256,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        ))

    
    model.add(Dense(1))
    return model
    


In [19]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)

In [20]:
model = bnn()
loss = keras.losses.MeanSquaredError()
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
    loss=loss,
    metrics=[keras.metrics.RootMeanSquaredError()],
    )

2023-03-03 10:33:19.763839: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 10:33:20.306755: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11290 MB memory:  -> device: 0, name: NVIDIA TITAN X (Pascal), pci bus id: 0000:05:00.0, compute capability: 6.1


In [21]:
model.fit(x_tra, y_tra, batch_size = BATCH_SIZE, epochs=EPOCHS, validation_data=(x_val, y_val))

Epoch 1/10000
Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


2023-03-03 10:33:22.654559: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f372c04bf20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-03 10:33:22.654595: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA TITAN X (Pascal), Compute Capability 6.1
2023-03-03 10:33:22.658389: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-03 10:33:22.758319: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


344/344 [==============================] - 46s 126ms/step - loss: 19.0383 - root_mean_squared_error: 3.9631 - val_loss: 21.9966 - val_root_mean_squared_error: 4.3108
Epoch 2/10000
344/344 [==============================] - 41s 120ms/step - loss: 22.5556 - root_mean_squared_error: 4.3614 - val_loss: 19.6156 - val_root_mean_squared_error: 4.0027
Epoch 3/10000
344/344 [==============================] - 44s 129ms/step - loss: 20.0254 - root_mean_squared_error: 4.0405 - val_loss: 19.6461 - val_root_mean_squared_error: 3.9811
Epoch 4/10000
344/344 [==============================] - 44s 129ms/step - loss: 15.6865 - root_mean_squared_error: 3.4437 - val_loss: 20.2542 - val_root_mean_squared_error: 4.0321
Epoch 5/10000
344/344 [==============================] - 44s 129ms/step - loss: 16.6371 - root_mean_squared_error: 3.5593 - val_loss: 16.1764 - val_root_mean_squared_error: 3.4841
Epoch 6/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.5351 - root_mean_squared_error:

344/344 [==============================] - 44s 129ms/step - loss: 12.2393 - root_mean_squared_error: 2.4763 - val_loss: 11.2474 - val_root_mean_squared_error: 2.2731
Epoch 47/10000
344/344 [==============================] - 44s 129ms/step - loss: 12.2368 - root_mean_squared_error: 2.4659 - val_loss: 11.4571 - val_root_mean_squared_error: 2.2864
Epoch 48/10000
344/344 [==============================] - 44s 129ms/step - loss: 12.2724 - root_mean_squared_error: 2.4628 - val_loss: 11.4582 - val_root_mean_squared_error: 2.2736
Epoch 49/10000
344/344 [==============================] - 44s 129ms/step - loss: 12.3242 - root_mean_squared_error: 2.4637 - val_loss: 11.1585 - val_root_mean_squared_error: 2.2236
Epoch 50/10000
344/344 [==============================] - 44s 129ms/step - loss: 12.3709 - root_mean_squared_error: 2.4623 - val_loss: 11.8804 - val_root_mean_squared_error: 2.3524
Epoch 51/10000
344/344 [==============================] - 44s 129ms/step - loss: 12.2925 - root_mean_squared_e

344/344 [==============================] - 44s 129ms/step - loss: 13.6165 - root_mean_squared_error: 2.4909 - val_loss: 13.0385 - val_root_mean_squared_error: 2.3639
Epoch 92/10000
344/344 [==============================] - 44s 129ms/step - loss: 13.6384 - root_mean_squared_error: 2.4962 - val_loss: 13.0423 - val_root_mean_squared_error: 2.3790
Epoch 93/10000
344/344 [==============================] - 44s 129ms/step - loss: 13.6356 - root_mean_squared_error: 2.4945 - val_loss: 12.5141 - val_root_mean_squared_error: 2.2765
Epoch 94/10000
344/344 [==============================] - 44s 129ms/step - loss: 13.5798 - root_mean_squared_error: 2.4802 - val_loss: 12.3967 - val_root_mean_squared_error: 2.2304
Epoch 95/10000
344/344 [==============================] - 44s 129ms/step - loss: 13.4380 - root_mean_squared_error: 2.4503 - val_loss: 12.6246 - val_root_mean_squared_error: 2.2789
Epoch 96/10000
344/344 [==============================] - 44s 129ms/step - loss: 13.6241 - root_mean_squared_e

...

344/344 [==============================] - 44s 129ms/step - loss: 14.3305 - root_mean_squared_error: 2.4592 - val_loss: 13.2300 - val_root_mean_squared_error: 2.2311
Epoch 182/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.2308 - root_mean_squared_error: 2.4365 - val_loss: 13.2625 - val_root_mean_squared_error: 2.2390
Epoch 183/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.4410 - root_mean_squared_error: 2.4782 - val_loss: 13.3742 - val_root_mean_squared_error: 2.2635
Epoch 184/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.3448 - root_mean_squared_error: 2.4642 - val_loss: 13.2598 - val_root_mean_squared_error: 2.2611
Epoch 185/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.3750 - root_mean_squared_error: 2.4685 - val_loss: 13.6960 - val_root_mean_squared_error: 2.3284
Epoch 186/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.2835 - root_mean_squa

344/344 [==============================] - 44s 129ms/step - loss: 14.2388 - root_mean_squared_error: 2.4398 - val_loss: 13.4739 - val_root_mean_squared_error: 2.2747
Epoch 227/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.2505 - root_mean_squared_error: 2.4445 - val_loss: 13.3126 - val_root_mean_squared_error: 2.2414
Epoch 228/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.2887 - root_mean_squared_error: 2.4499 - val_loss: 13.4734 - val_root_mean_squared_error: 2.2756
Epoch 229/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.3096 - root_mean_squared_error: 2.4442 - val_loss: 13.3655 - val_root_mean_squared_error: 2.2375
Epoch 230/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.2584 - root_mean_squared_error: 2.4425 - val_loss: 13.7220 - val_root_mean_squared_error: 2.3191
Epoch 231/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.3849 - root_mean_squa

344/344 [==============================] - 44s 129ms/step - loss: 14.4102 - root_mean_squared_error: 2.4680 - val_loss: 13.2622 - val_root_mean_squared_error: 2.2499
Epoch 272/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.2664 - root_mean_squared_error: 2.4502 - val_loss: 13.3522 - val_root_mean_squared_error: 2.2537
Epoch 273/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.2233 - root_mean_squared_error: 2.4378 - val_loss: 13.8862 - val_root_mean_squared_error: 2.3703
Epoch 274/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.2591 - root_mean_squared_error: 2.4523 - val_loss: 13.5468 - val_root_mean_squared_error: 2.2932
Epoch 275/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.1238 - root_mean_squared_error: 2.4156 - val_loss: 13.2161 - val_root_mean_squared_error: 2.2328
Epoch 276/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.2285 - root_mean_squa

344/344 [==============================] - 44s 129ms/step - loss: 14.0451 - root_mean_squared_error: 2.4162 - val_loss: 13.4124 - val_root_mean_squared_error: 2.2616
Epoch 317/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.2397 - root_mean_squared_error: 2.4468 - val_loss: 13.1285 - val_root_mean_squared_error: 2.2214
Epoch 318/10000
344/344 [==============================] - 44s 129ms/step - loss: 14.2460 - root_mean_squared_error: 2.4524 - val_loss: 13.1732 - val_root_mean_squared_error: 2.2246
Epoch 319/10000
227/344 [==================>...........] - ETA: 14s - loss: 14.0051 - root_mean_squared_error: 2.4070

KeyboardInterrupt: 

In [ ]:
model.evaluate(x_tra, y_tra)

In [ ]:
predicted = []
iterations = 5
for i in range(iterations):
    predicted.append(model.predict(e_test_x))
predicted = np.concatenate(predicted, axis=1)


In [ ]:
predicted

In [ ]:
prediction_mean = np.mean(predicted, axis=1).tolist()
prediction_min = np.min(predicted, axis=1).tolist()
prediction_max = np.max(predicted, axis=1).tolist()
prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

In [ ]:
v = model.evaluate(x_tra, y_tra)
t = model.evaluate(e_test_x, e_test_y)
line_test = [str(v), str(t)]
for idx in range(len(e_test_x)):
    s = e_test_name[idx] + " " +f"Predictions mean: {round(prediction_mean[idx], 2)}, " + f"min: {round(prediction_min[idx], 2)}, " + f"max: {round(prediction_max[idx], 2)}, " + f"range: {round(prediction_range[idx], 2)} - " + f"Actual: {e_test_y[idx]}"
    line_test.append(s)


In [ ]:
#model.save('bnn_density_d.h5')

In [ ]:
with open('bnn_density_d.txt', 'w') as f:
    for line in line_test:
        f.write(line)
        f.write('\n')

In [ ]:
line_test